## Llama-Index Agents + Custom Evaluations

In [1]:
# Setup OpenAI Agent
from llama_index.agent import OpenAIAgent
import openai
openai.api_key = '...'

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

In [3]:
# Import and initialize our tool spec
from llama_hub.tools.yelp.base import YelpToolSpec
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

tool_spec = YelpToolSpec(api_key='...', client_id='...')

In [4]:
def llm_standalone(prompt):
    return openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are a question and answer bot, and you answer concisely."},
            {"role": "user", "content": prompt}
        ]
    )["choices"][0]["message"]["content"]

In [5]:
llm_standalone("what are good restaurants in toronto?")

'There are numerous good restaurants in Toronto, some popular ones include Canoe, Richmond Station, Alo Restaurant, Pai, and Bar Isabel.'

In [6]:
# Create the Agent with our tools
tools = tool_spec.to_tool_list()
agent = OpenAIAgent.from_tools(
    [
        *LoadAndSearchToolSpec.from_defaults(tools[0]).to_tool_list(),
        *LoadAndSearchToolSpec.from_defaults(tools[1]).to_tool_list()
    ],
    verbose=True
)

In [7]:
agent.chat("what are good restaurants in toronto")

=== Calling Function ===
Calling function: business_search with args: {
  "location": "Toronto",
  "term": "restaurants"
}
Got output: Content loaded! You can now search the information using read_business_search
=== Calling Function ===
Calling function: read_business_search with args: {
  "query": "good restaurants in Toronto"
}
Got output: 
Based on the context information, some good restaurants in Toronto include Ristorante Sotto Sotto, The Oxley, KINKA IZAKAYA ORIGINAL, and Dirty Food Eatery.


AgentChatResponse(response='Some good restaurants in Toronto include Ristorante Sotto Sotto, The Oxley, KINKA IZAKAYA ORIGINAL, and Dirty Food Eatery.', sources=[ToolOutput(content='Content loaded! You can now search the information using read_business_search', tool_name='business_search', raw_input={'args': (), 'kwargs': {'location': 'Toronto', 'term': 'restaurants'}}, raw_output='Content loaded! You can now search the information using read_business_search'), ToolOutput(content='\nBased on the context information, some good restaurants in Toronto include Ristorante Sotto Sotto, The Oxley, KINKA IZAKAYA ORIGINAL, and Dirty Food Eatery.', tool_name='read_business_search', raw_input={'args': (), 'kwargs': {'query': 'good restaurants in Toronto'}}, raw_output='\nBased on the context information, some good restaurants in Toronto include Ristorante Sotto Sotto, The Oxley, KINKA IZAKAYA ORIGINAL, and Dirty Food Eatery.')])

In [8]:
from trulens_eval import Feedback, OpenAI, Tru, TruBasicApp, TruLlama

tru = Tru()

class OpenAI_custom(OpenAI):
    def definitive(self, response: str) -> float:

        return float(openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "Your job is to rate how definitive the following text is on a scale of 1 to 10. Respond with the number only."},
            {"role": "user", "content": response}
        ]
    )["choices"][0]["message"]["content"]) / 10

custom = OpenAI_custom()
definitive = Feedback(custom.definitive).on_output()

✅ In definitive, input response will be set to *.__record__.main_output or `Select.RecordOutput` .


In [9]:
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.23:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [10]:
standalone_app = TruBasicApp(llm_standalone, app_id="OpenAIChatCompletion", feedbacks=[definitive])

✅ app OpenAIChatCompletion -> default.sqlite
✅ feedback def. feedback_definition_hash_491763e5a520c2e026ec536169856e41 -> default.sqlite


In [11]:
yelp_app = TruLlama(agent,
    app_id='YelpAgent',
    feedbacks=[definitive])

✅ app YelpAgent -> default.sqlite
✅ feedback def. feedback_definition_hash_491763e5a520c2e026ec536169856e41 -> default.sqlite


In [12]:
standalone_app.call_with_record("Where is the best poutine in montreal?")

("The best poutine in Montreal is subjective and can vary based on personal taste. However, popular places for poutine include La Banquise, Patati Patata, and Ma'am Bolduc.",
 Record(record_id='record_hash_c03260cb30bc7242438197370d0bf658', app_id='OpenAIChatCompletion', cost=Cost(n_requests=1, n_successful_requests=1, n_classes=0, n_tokens=76, n_prompt_tokens=36, n_completion_tokens=40, cost=0.000134), perf=Perf(start_time=datetime.datetime(2023, 7, 20, 12, 46, 37, 792108), end_time=datetime.datetime(2023, 7, 20, 12, 46, 39, 963763)), ts=datetime.datetime(2023, 7, 20, 12, 46, 39, 963812), tags='-', main_input='Where is the best poutine in montreal?', main_output="The best poutine in Montreal is subjective and can vary based on personal taste. However, popular places for poutine include La Banquise, Patati Patata, and Ma'am Bolduc.", main_error='None', calls=[RecordAppCall(stack=(RecordAppCallMethod(path=JSONPath().app, method=Method(obj=Obj(cls=trulens_eval.tru_basic_app.TruWrapperApp

✅ record record_hash_c03260cb30bc7242438197370d0bf658 from OpenAIChatCompletion -> default.sqlite

In [13]:
yelp_app.query("Where is the best poutine in montreal?")


✅ feedback feedback_result_hash_be9508abac93e976a34d1358d7c906bd on record_hash_c03260cb30bc7242438197370d0bf658 -> default.sqlite
=== Calling Function ===
Calling function: business_search with args: {
  "location": "Montreal",
  "term": "poutine"
}
Got output: Content loaded! You can now search the information using read_business_search
=== Calling Function ===
Calling function: read_business_search with args: {
  "query": "best poutine in Montreal"
}
Got output: 
Based on the context information provided, the best poutine in Montreal appears to be Poutineville, which has a 4.5 star rating and 40 reviews.


Response(response='The best poutine in Montreal is at Poutineville. It has a rating of 4.5 stars and 40 reviews.', source_nodes=[], metadata=None)

✅ record record_hash_685f548cc22064d405eead007b13f764 from YelpAgent -> default.sqlite
✅ feedback feedback_result_hash_c68db2c051ed4e681eab575108367432 on record_hash_685f548cc22064d405eead007b13f764 -> default.sqlite
